In [146]:
import tensorflow as tf
import numpy as np
import pandas as pd

from tensorflow.keras.layers import ( Input, Embedding, Flatten, Dot, Add, Concatenate, 
        Dense, Dropout, Activation, Multiply )
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2

In [147]:
ratings = pd.read_csv('rratings.csv', sep=',',encoding='latin-1')
ratings.head()

,userId,routeId,rating
0,cjsysx7la00en0836zceujv79,cjsyxzoer01j908824mfq6lg6,1.2
1,cjsysx7la00en0836zceujv79,cjsyxzo7b01ba0882kjs0ta26,4.0
2,cjsysx7la00en0836zceujv79,cjsyxzo7m01c10882s8gmo2rt,2.7
3,cjsysx7la00en0836zceujv79,cjsyxzodf01h40882qzwjsdqx,1.2
4,cjsysx7la00en0836zceujv79,cjsyxzod601ga088290hvk8fb,3.5


In [148]:
u_unique = ratings.userId.unique()
user2Id = { userId : index for index, userId in enumerate(u_unique)}
ratings.userId = ratings.userId.apply(lambda x: user2Id[x])

r_unique = ratings.routeId.unique()
route2Id = { routeId : index for index, routeId in enumerate(r_unique)}
ratings.routeId = ratings.routeId.apply(lambda x: route2Id[x])

In [149]:
n_users = ratings.userId.nunique()
n_routes = ratings.routeId.nunique()
n_users, n_routes

(103, 376)

In [150]:
n_factors = 32

In [151]:
np.random.seed = 42

In [75]:
msk = np.random.rand(len(ratings)) < 0.8

In [126]:
trn = ratings[msk]
val = ratings[~msk]
val.shape

(204, 3)

In [127]:
def embedding_input(name, n_in, n_out, reg):
    inp = Input(shape=(1,), dtype='int64', name=name)
    return inp, Embedding(n_in, n_out, input_length=1, embeddings_regularizer=l2(reg))(inp)

# Neural Network Model

In [143]:
user_in, u = embedding_input('user_in', n_users, n_factors, 1e-3)
route_in, r = embedding_input('route_in', n_routes, n_factors, 1e-3)

In [144]:
x = Concatenate()([u, r])
x = Flatten()(x)
x = Dense(units=(n_factors * 2), activation='relu')(x)
x = Dropout(rate=0.75)(x)
x = Dense(1)(x)
model = tf.keras.Model(inputs=[user_in, route_in], outputs=x)
model.compile(optimizer=Adam(lr=0.001), loss='mse')

In [145]:
model.fit([trn.userId, trn.routeId], trn.rating, batch_size=64, epochs=10,
          validation_data=([val.userId, val.routeId], val.rating))

Train on 814 samples, validate on 204 samples
Epoch 1/10
814/814 [==============================] - 5s 6ms/sample - loss: 5.8005 - val_loss: 5.7161
Epoch 2/10
814/814 [==============================] - 0s 201us/sample - loss: 4.8342 - val_loss: 4.5717
Epoch 3/10
814/814 [==============================] - 0s 291us/sample - loss: 3.5697 - val_loss: 3.1520
Epoch 4/10
814/814 [==============================] - 0s 293us/sample - loss: 2.3898 - val_loss: 1.9725
Epoch 5/10
814/814 [==============================] - 0s 290us/sample - loss: 1.7087 - val_loss: 1.5927
Epoch 6/10
814/814 [==============================] - 0s 321us/sample - loss: 1.5373 - val_loss: 1.5776
Epoch 7/10
814/814 [==============================] - 0s 453us/sample - loss: 1.3750 - val_loss: 1.6074
Epoch 8/10
814/814 [==============================] - 0s 288us/sample - loss: 1.3361 - val_loss: 1.6780
Epoch 9/10
814/814 [==============================] - 0s 408us/sample - loss: 1.2062 - val_loss: 1.7519
Epoch 10/10
814/814 

In [152]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
user_in (InputLayer)            (None, 1)            0                                            
__________________________________________________________________________________________________
route_in (InputLayer)           (None, 1)            0                                            
__________________________________________________________________________________________________
embedding_46 (Embedding)        (None, 1, 32)        3296        user_in[0][0]                    
__________________________________________________________________________________________________
embedding_47 (Embedding)        (None, 1, 32)        12032       route_in[0][0]                   
__________________________________________________________________________________________________
concatenat

In [153]:
model.save('my_model.h5')